!pip install unstructured_inference

!pip install pytesseract

!pip install tesseract

!sudo apt update

!sudo apt install tesseract-ocr

!sudo apt install libtesseract-dev

---

Load the documents, one per item in the list per pdf

In [17]:
from langchain.document_loaders import DirectoryLoader

loader = DirectoryLoader("/home/gonza/gonza/Financial_Chatbot_Advisor/dataset",show_progress=True)
docs = loader.load()

 40%|████      | 401/998 [3:05:13<3:39:18, 22.04s/it]

PSEOF: Unexpected EOF

In [ ]:
len(docs)

Instanciate the splitter

In [18]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
r_splitter = RecursiveCharacterTextSplitter(
    chunk_size=2000,
    chunk_overlap=100 ,
    separators=["\n\n", "\n", "(?<=\. )", " ", ""],
    length_function=len
    )

Do the spliting

In [ ]:
splits = r_splitter.split_documents(docs)
len(splits)

11247

Load it to the Chroma Vectorstore

In [9]:
from langchain.document_loaders import DirectoryLoader

loader = DirectoryLoader("/home/gonza/gonza/Financial_Chatbot_Advisor/dataset_prueba_5",show_progress=True)
docs = loader.load()

100%|██████████| 5/5 [02:24<00:00, 28.92s/it]


In [13]:
docs[0].metadata

{'source': '/home/gonza/gonza/Financial_Chatbot_Advisor/dataset_prueba_5/AMEX_ATRS_2012.pdf'}

In [15]:
docs[0].metadata

dict

In [ ]:
split -> sacar metadata

In [1]:
import pandas as pd
df=pd.read_csv('/home/gonza/gonza/Financial_Chatbot_Advisor/git/FinancialAdvisorChatbot/dataset_v2.csv')

In [2]:
df.columns

Index(['name', 'exchange', 'year', 'document', 'Name', 'Market Cap', 'Sector',
       'Industry', 'Country', '_split_id', 'page', '_split_overlap'],
      dtype='object')

In [3]:
splits=df.document.tolist()

In [4]:
metadata=df.drop(columns='document', axis=1)

In [8]:
metadata=metadata.to_dict(orient='records')

In [9]:
type(metadata)

list

In [26]:
splits_strings = [str(elemento) for elemento in splits]


In [15]:
from langchain.embeddings import HuggingFaceEmbeddings, SentenceTransformerEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="multi-qa-MiniLM-L6-cos-v1")

In [28]:
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings, SentenceTransformerEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="multi-qa-MiniLM-L6-cos-v1")
db_summarized = Chroma.from_texts(
    texts=splits_strings,
    metadatas=metadata,
    persist_directory="chroma_full",
    embedding=embeddings,
    collection_name='dataset_by_chunk'
)

In [8]:
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings, SentenceTransformerEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="multi-qa-MiniLM-L6-cos-v1")
#sentence_transformer_ef = embedding_functions.SentenceTransformerEmbeddingFunction(model_name="multi-qa-MiniLM-L6-cos-v1")
#embeddings = SentenceTransformerEmbeddings(model_name="multi-qa-MiniLM-L6-cos-v1")

db = Chroma.from_documents(splits,persist_directory="chroma_30", embedding=embeddings)
db.persist()

Try if it worked

In [7]:
db.get()['documents'][0]

'UNITED STATES SECURITIES AND EXCHANGE COMMISSION Washington, D.C. 20549\n\nFORM 10-K\n\n[X] ANNUAL REPORT PURSUANT TO SECTION 13 OR 15 (D) OF THE SECURITIES EXCHANGE ACT OF 1934 FOR THE FISCAL YEAR ENDED DECEMBER 31, 2012\n\n[ ] TRANSITION REPORT PURSUANT TO SECTION 13 OR 15 (d) OF THE SECURITIES EXCHANGE ACT OF 1934 For transition period from __________ to __________\n\nCommission file number 1-32302\n\nANTARES PHARMA, INC. (Exact name of registrant as specified in its charter)\n\nA Delaware corporation\n\nI.R.S. Employer Identification No. 41-1350192\n\n100 Princeton South, Suite 300, Ewing, NJ 08628\n\nRegistrant’s telephone number, including area code: (609) 359-3020\n\nSecurities registered pursuant to section 12(b) of the Act:\n\nTitle of each class Common Stock\n\nName of each exchange on which registered NASDAQ Capital Market\n\nSecurities registered pursuant to section 12(g) of the Act: None\n\nIndicate by check mark if the registrant is a well-known seasoned issuer, as defin